In [87]:
import yfinance as yf
import pandas as pd
from datetime import datetime
from datetime import timedelta

In [132]:
empresas = ['AMER3_2', 'MGLU3_2', 'PETR4_2']

In [133]:
# for empresa in empresas:
#     dados = pd.read_csv(empresa + '.csv', index_col='Unnamed: 0')

#     dados['data_hora'] = dados['data'] + ' ' + dados['hora']
#     dados['data_hora'] = dados['data_hora'].str.replace('.', '-')

#     dados['data_hora'] = dados['data_hora']+":00"

#     colunas = ['data_hora', 'url', 'titulo', 'texto']
#     dados_tratados = dados[colunas]

#     # print(dados_tratados.head())
#     dados_tratados.to_csv(empresa + '.csv')


In [189]:
def getCsv(empresa):
    dados = pd.read_csv(empresa + '.csv', index_col='Unnamed: 0')
    
    if empresa == 'AMER3_2':
        dados['data_hora'] = pd.to_datetime(dados['data_hora'], dayfirst=True, format='%Y-%m-%d %H:%M:%S')
    else:
        dados['data_hora'] = pd.to_datetime(dados['data_hora'], dayfirst=True, format='%d-%m-%Y %H:%M:%S')

    return dados.sort_values('data_hora')

ERROR! Session/line number was not unique in database. History logging moved to new session 135


###################################################################################

In [216]:
def getDadosBolsa(empresa, dataMax, dataMin):

    dados = yf.Ticker(empresa + '.SA')
    dados = dados.history(start=(dataMin - timedelta(days=7)), end=(dataMax + timedelta(days=7)), interval='1d')
    dados.reset_index(inplace=True)
    dados['data_hora'] = dados['Date']
    dados.drop(columns=['Date'], inplace=True)
    dados.set_index('data_hora', inplace=True)
    return dados

In [217]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25 entries, 24 to 0
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   data_hora              25 non-null     datetime64[ns]
 1   url                    25 non-null     object        
 2   titulo                 25 non-null     object        
 3   texto                  25 non-null     object        
 4   aberturaDiaAnterior    25 non-null     float64       
 5   fechamentoDiaAnterior  25 non-null     float64       
 6   aberturaDiaAtual       25 non-null     float64       
 7   fechamentoDiaAtual     25 non-null     float64       
 8   aberturaDiaSeguinte    25 non-null     float64       
 9   fechamentoDiaSeguinte  25 non-null     float64       
 10  abertura2Dias          25 non-null     float64       
 11  fechamento2Dias        25 non-null     float64       
dtypes: datetime64[ns](1), float64(8), object(3)
memory usage: 2.5+ KB


In [210]:
def getDataMaisProxima(empresa, data):

    datas = {
                abs(data.timestamp() - date.timestamp()) : date for date in empresa.index}
    
    dataMaisProxima = datas[min(datas.keys())]

    return dataMaisProxima - timedelta(days=1), dataMaisProxima, dataMaisProxima + timedelta(days=1), dataMaisProxima + timedelta(days=2)

In [229]:
def getValores(dados, data):
    try:
        return dados.iloc[dados.index.get_loc(data)]

    except KeyError:
        return getValores(dados, data + timedelta(days=1))

def montaDictValores(valores):
    return {
        'Abertura': valores.Open,
        'Fechamento': valores.Close,
    }

def getValoresData(dados, data):
    print(getValores(dados, data))
    return montaDictValores(getValores(dados, data))

In [230]:
datetime.today().date()

datetime.date(2023, 10, 7)

In [231]:
def extraiValoresBolsa(empresa):
    df = getCsv(empresa)
    if '_' in empresa:
        dados_bolsa = getDadosBolsa(empresa[:-2], df['data_hora'].max(), df['data_hora'].min())
    else:
        dados_bolsa = getDadosBolsa(empresa, df['data_hora'].max(), df['data_hora'].min())

    aberturaDiaAnterior = []
    aberturaDiaAtual = []
    aberturaDiaSeguinte = []
    abertura2Dias = []

    fechamentoDiaAnterior = []
    fechamentoDiaAtual = []
    fechamentoDiaSeguinte = []
    fechamento2Dias = []

    print(df.shape)
    i = 0
    for data in df['data_hora']:
        
        diaAnterior, dataAtual, dataSeguinte, dia2 = getDataMaisProxima(dados_bolsa, data)

        valores = getValoresData(dados_bolsa, diaAnterior)
        aberturaDiaAnterior.append(valores['Abertura'])
        fechamentoDiaAnterior.append(valores['Fechamento'])

        valores = getValoresData(dados_bolsa, dataAtual)
        aberturaDiaAtual.append(valores['Abertura'])
        fechamentoDiaAtual.append(valores['Fechamento'])

        # print(dataSeguinte)
        valores = getValoresData(dados_bolsa, dataSeguinte)
        # print(valores)
        aberturaDiaSeguinte.append(valores['Abertura'])
        fechamentoDiaSeguinte.append(valores['Fechamento'])
        # print(i)
        # i += 1
        
        valores = getValoresData(dados_bolsa, dia2)
        abertura2Dias.append(valores['Abertura'])
        fechamento2Dias.append(valores['Fechamento'])

        

    print('cheguei aqui') 
    
    df['aberturaDiaAnterior'] = aberturaDiaAnterior
    df['fechamentoDiaAnterior'] = fechamentoDiaAnterior

    df['aberturaDiaAtual'] = aberturaDiaAtual
    df['fechamentoDiaAtual'] = fechamentoDiaAtual

    df['aberturaDiaSeguinte'] = aberturaDiaSeguinte
    df['fechamentoDiaSeguinte'] = fechamentoDiaSeguinte

    df['abertura2Dias'] = abertura2Dias
    df['fechamento2Dias'] = fechamento2Dias

    return df

In [ ]:
for empresa in empresas:
    print(empresa)
    df = extraiValoresBolsa(empresa)
    df.to_csv(empresa + '.csv')

In [ ]:
dfm = extraiValoresBolsa(empresas[1])
dfm.head()  